## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-18-47-49 +0000,bbc,Historic lodge destroyed in Grand Canyon blaze,https://www.bbc.com/news/articles/c33527p2nzeo
1,2025-07-14-18-40-00 +0000,nyt,Clashes between Bedouin and Druse in Syria Kil...,https://www.nytimes.com/2025/07/14/world/middl...
2,2025-07-14-18-36-14 +0000,latimes,Trump officials vow to intensify immigration r...,https://www.latimes.com/california/story/2025-...
3,2025-07-14-18-24-51 +0000,nyt,Zelensky Announces Plan for Major Government S...,https://www.nytimes.com/2025/07/14/world/europ...
4,2025-07-14-18-05-18 +0000,nyt,Companies in Europe Are Frozen as Tariff Uncer...,https://www.nytimes.com/2025/07/14/business/ta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,16
34,ukraine,6
74,trade,6
79,will,5
25,public,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,50
58,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,47
5,2025-07-14-18-04-03 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,40
30,2025-07-14-14-18-31 +0000,nyt,Trump Reluctantly Comes Around to Backing Ukra...,https://www.nytimes.com/2025/07/14/us/politics...,33
6,2025-07-14-17-28-18 +0000,nyt,Trump Promises to Help Europe Speed Weapons to...,https://www.nytimes.com/2025/07/14/us/politics...,33


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,50,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
5,24,2025-07-14-18-04-03 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
46,20,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
17,17,2025-07-14-16-08-41 +0000,bbc,Dozens killed in clashes between Druze and Bed...,https://www.bbc.com/news/articles/cgmwp70pwmyo
52,17,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
24,15,2025-07-14-14-55-07 +0000,bbc,"Boeing fuel switches safe, regulator says afte...",https://www.bbc.com/news/articles/ce9xpgnx3vdo
58,15,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
66,15,2025-07-13-20-15-07 +0000,nyt,South African President Opens Corruption Inqui...,https://www.nytimes.com/2025/07/13/world/afric...
2,14,2025-07-14-18-36-14 +0000,latimes,Trump officials vow to intensify immigration r...,https://www.latimes.com/california/story/2025-...
11,12,2025-07-14-16-50-44 +0000,latimes,Rachel Scott and James Longman will co-anchor ...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
